In [ ]:
!pip install kaggle
import os
from torchvision import datasets, transforms
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
import torch.optim.lr_scheduler as lr_scheduler
from PIL import Image
import os
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"
!kaggle competitions download -c vub-ml-project-2024-animal-classification
!mkdir -p /content/drive/MyDrive/kaggle
!unzip vub-ml-project-2024-animal-classification.zip -d /content/

In [ ]:
# hyperparams
num_epochs = 1
lr = 0.01
num_classes = 12
num_channels = 3
batch_size = 64

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Define directories
train_dir = "/content/vub-ml-project-2024-animal-classification/train"
test_dir = "/content/vub-ml-project-2024-animal-classification/test"

# Create datasets using ImageFolder
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
#test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

# Split training dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 32  # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
#test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the class-to-index mapping
print("Class-to-Index Mapping:", train_dataset.dataset.class_to_idx)


In [ ]:
def test(model, testDataLoader, device):
    model.eval().to(device)
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in testDataLoader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = (correct / total) * 100
    return acc

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=4, stride=4),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=4)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512 * 12 * 12, 1024),  # Adjust input dimensions accordingly
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 12),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)

        return x



In [ ]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
#train_loader, val_loader, test_loader = data_loader(BATCH_SIZE)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


sample_input = torch.rand((1, num_channels, 32, 32)).to(device)


total_step = len(train_loader)

for epoch in range(num_epochs):
    model.train()
    epoch_losses = []
    epoch_accuracies = []
    total_correct = 0
    total_samples = 0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        _, predicted = outputs.max(1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / labels.size(0)

        epoch_losses.append(loss.item())
        epoch_accuracies.append(accuracy)
        total_correct += correct
        total_samples += labels.size(0)

        if (i + 1) % 100 == 0:
            print(f'LR: {lr}, Epoch: [{epoch+1}/{num_epochs}], Step: [{i+1}/{total_step}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')

    # Validation accuracy
    val_accuracy = test(model, val_loader, device)
    train_accuracy = total_correct / total_samples
    #test_accuracy = test(model, test_loader, device)
    #writer.add_scalar('Validation/Accuracy', val_accuracy, epoch)
    #writer.add_scalar('Test/Accuracy', test_accuracy, epoch)

    #print(f'Epoch [{epoch+1}/{num_epochs}] Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}')



